# Model

Construção do modelo após exploração e análise do dataset, contido em **data_exploration.ipynb**

- - -
### Pipeline
    - Load Data
        - Carregar os dados processados anteriormente
    
    - Transformação das variáveis contínuas em categóricas
        - Utilização do modelo de agrupamento KMEANS para busca dos limiares categóricos
    
    - Análise das categorias
        - Analise de quartils para validação da transformação categórica
    
    - Teste de Shapiro
        - A amostragem tem distribuição gaussiana?
    
    - Modelo Gaussian NB
        - Dado a amostragem, qual a categoria o processo se encaixa?
    
    - Resultados
        - score do modelo

- - -


### 1) Bibliotecas

In [35]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

### 2) Load processed data

In [36]:
data_path = "../data/"
csv_list = ["x_test_pos.csv", "x_train_pos.csv", "y_test_pos.csv", "y_train_pos.csv"]


x_test = pd.read_csv(data_path+csv_list[0])
x_train = pd.read_csv(data_path+csv_list[1])
y_test = pd.read_csv(data_path + csv_list[2])
y_train = pd.read_csv(data_path + csv_list[3])

# 3) Transformando variaveis contínuas em categóricas
- - -
Para transformação das variáveis utilizo o modelo **KMeans** fixando 2 clusters, desta forma temos 2 valores que definirão 3 categorias:

> Categoria 1: y < Cluster_1

> Categoria 2: Cluster_1 < y < Cluster_2

> Categoria 3: y > Cluster_2

In [37]:
X = np.array(x_train)
test_x = np.array(x_test)
y = np.array(y_train)
test_y = np.array(y_test)

kmeans = KMeans(n_clusters=2)
kmeans.fit(y)
kmeans.cluster_centers_

array([[-63.55134324],
       [ 85.71634138]])

Note que os valores das clusters são: **[-63.5, 85.71]**

## 3.1) Resultado das categorias

**Categoria 0**: y < -63.5

**Categoria 1**: -63.5 < y < 85.71

**Categoria 2**: y > 85.71
- - -

## 3.2) Análise de Quartil

Para verificar se a técnica faz sentido, faço a análise de quartil e espera-se que os quartils estejam próximos dos valores do centro das Clusters


In [38]:
cat_1 = y_train.quantile(.25)
cat_2 = y_train.quantile(.50)
cat_3 = y_train.quantile(.75)
cat_4 = y_train.quantile(.85)
print(r'Quartil {}: {} '.format(25,cat_1[0]))
print(r'Quartil {}: {} '.format(50,cat_2[0]))
print(r'Qualtil {}: {} '.format(75,cat_3[0]))
print(r'Qualtil {}: {} '.format(90,cat_4[0]))

Quartil 25: -60.60987262721621 
Quartil 50: 3.816762635537936 
Qualtil 75: 61.731313282480265 
Qualtil 90: 98.89486312899137 


- O centro da cluster 0 está próxima ao quartil 25
   
- O centro da cluster 1 está entre os quartils 75 e 90

Logo, a técnica de utilizar os centros das clusters como limiares de categoria está satisfatório. 

A *Categoria 0* tem limiar próximo ao quartil 25.

A *Categoria 1* está entre os quartils 25 e 75. 

Por fim, a *Categoria 2* está próxima ao quartil 90.
- - -
## 3.3) Transformando Y em classes

Faz a substituição dos valores contínuos em Categorias definidas pelos valores das clusters calculados anteriormente.

In [39]:

def to_categorical(data, cat_1, cat_3):
    bag = []
    for i in data:
        if i < cat_1[0]:
            bag.append(1)
            pass
        elif  i < cat_3[0] and i > cat_1[0]:
            bag.append(2)
            pass
        elif i > cat_3[0]:
            bag.append(3)
            pass
    return bag
y_cat = to_categorical(y,kmeans.cluster_centers_[0], kmeans.cluster_centers_[1]) 
y_test_cat = to_categorical(test_y,kmeans.cluster_centers_[0], kmeans.cluster_centers_[1])

# 4) Modelo Gaussian Naive Bayes

Uma vez que *y* é categórico e os valores *X* não apresentam uma correlação linear *(Observada no Heatmap)*, o modelo Gaussian Naive Bayes se ajusta bem ao problema, pois o modelo trata as variável de maneira independente.

**Para utilizar esse modelo precisamos testar se a distribuição dos dados é Normal (Gaussiana).**
- - -
## 4.1) Teste de Shapiro

O teste de Shapiro diz se amostragem vem de uma distribuição normal.

Quanto mais próximo o valor *statistic* for de 1 ou *pvalue* > 0.05, há mais chances da amostra vir de uma distribuição Gaussiana.

In [40]:
a = np.array(x_train)
stats.shapiro(a)

ShapiroResult(statistic=0.9979251623153687, pvalue=0.054054636508226395)

O resultado indica que provavelmente a distribuição da amostragem X é Gaussiana, assim podemos utilizar o modelo GaussianNB.

**Uma vez que a distribuição é normal, não será necessária a transformação de escala da amostragem**
- - -
## 4.2) GaussianNB

- X: Amostras
- y_cat: Valores Target Categóricos
- y_pred: Predict em relação a amostragem teste

In [41]:
model = GaussianNB()
model.fit(X,y_cat)
y_pred = model.predict(test_x)

### 4.3) Resultados

- Score do modelo: 80%

Dado uma nova amostragem, o modelo tem precisão de 80% ao inferir a categoria.

In [42]:
model.score(test_x,y_test_cat)

0.84

## 4.4) Mapeando os Nomes

Transforma os nomes categóricos em "Normal", "Amarelo" e "Vermelho"

Adiciona o resultado no dataframe teste

In [43]:
translate = {1:'Normal', 2:'Amarelo', 3:'Vermelho'}

x_test['predict'] = y_pred
x_test['Y'] = y_test_cat

In [44]:
results = x_test.replace(translate)
results

,0,1,2,3,4,5,6,7,8,9,predict,Y
0,-2.818653,0.827178,0.862119,-0.524020,0.555806,-0.299716,-0.857083,0.667588,0.941755,-0.646962,Amarelo,Amarelo
1,2.297886,1.276248,-0.470093,-0.719206,-0.158068,-0.548281,-0.695010,-0.508080,0.845107,-0.364186,Amarelo,Amarelo
2,0.104152,-1.132729,0.995184,1.503805,-0.357791,0.439251,0.976177,0.548557,0.279874,0.460544,Amarelo,Amarelo
3,-1.815600,0.738951,0.334091,-1.136626,0.336871,-1.930223,0.940455,-0.078442,-0.688541,-0.723060,Amarelo,Amarelo
4,0.814325,1.214060,-1.611950,0.382659,0.845913,0.323513,0.177591,0.022726,1.109274,0.746312,Amarelo,Amarelo
5,0.849320,-0.717197,0.627196,-0.844609,0.125487,0.019711,0.331285,1.392872,-0.365801,2.275108,Amarelo,Amarelo
6,-0.410899,-0.863695,0.609106,1.261957,1.245524,-0.604518,0.209276,0.068868,0.920623,0.868650,Amarelo,Vermelho
7,0.112487,0.533366,0.265740,0.510694,-0.625533,1.190732,0.670157,-0.536905,-0.875099,-1.382035,Amarelo,Amarelo
8,-2.619281,0.586165,-0.762491,0.639771,2.156025,1.085494,-1.579599,-0.084384,-0.106784,-0.871303,Amarelo,Vermelho
9,0.708526,-0.177045,-1.707745,0.493568,-2.266885,0.959283,0.997393,0.123729,-0.698709,0.290321,Normal,Normal


# 4.5) Score do Modelo

In [45]:
j = 0
for i in range(len(results)):
    
    if results['predict'][i]==results['Y'][i]:
        j+=1
    else:
        pass
print('Score:', j/len(results))

Score: 0.84
